## LANGAUGE DETECTION

The most important part of training a language detection model is data. The more data you have about every language, the more accurate your model will perform in real-time. The dataset that I am using is collected from Kaggle, which contains data about 

In [1]:
import pandas as pd
df = pd.read_csv("dataset.csv")
df.head()

,Text,language
0,klement gottwaldi surnukeha palsameeriti ning ...,Estonian
1,sebes joseph pereira thomas på eng the jesuit...,Swedish
2,ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...,Thai
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...,Tamil
4,de spons behoort tot het geslacht haliclona en...,Dutch


check for null values

In [2]:
df.isna().sum()

Text        0
language    0
dtype: int64

In [13]:
df.language.value_counts()

Estonian      1000
Swedish       1000
English       1000
Russian       1000
Romanian      1000
Persian       1000
Pushto        1000
Spanish       1000
Hindi         1000
Korean        1000
Chinese       1000
French        1000
Portugese     1000
Indonesian    1000
Urdu          1000
Latin         1000
Turkish       1000
Japanese      1000
Dutch         1000
Tamil         1000
Thai          1000
Arabic        1000
Name: language, dtype: int64

A BALANCED DATASET

In [12]:
len(df.language.unique())

22

LANGUAGE DETECTION MODEL

In [14]:
import numpy as np

In [17]:
x = np.array(df.Text)
y = np.array(df.language)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [19]:
cv = CountVectorizer()
X = cv.fit_transform(x)
X[:10]

<10x277720 sparse matrix of type '<class 'numpy.int64'>'
	with 302 stored elements in Compressed Sparse Row format>

lets try hyper parameter tuning